In [ ]:
import sys
sys.path.insert(0, '../input/siim-acr-pneumothorax-segmentation')
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from mask_functions import *

In [ ]:
DATA_PATH = Path("../input/siim-acr-pneumothorax-segmentation-data/pneumothorax")
print(DATA_PATH)

In [ ]:
print(os.listdir(DATA_PATH))

In [ ]:
TRAIN = DATA_PATH / "dicom-images-train"
TEST = DATA_PATH / "dicom-images-test"
MASK = DATA_PATH / "train-rle.csv"

In [ ]:
mask_rle_df = pd.read_csv(MASK)
mask_rle_df.describe()

So there are total 11,582 rows of mask data and 10,675 unique ImageId. But there are only 3,286 images with EncodedPixels which means contain pneumothorax segmentation masks and 8,296 images without it (EncodedPixels == -1)

In [ ]:
mask_rle_df.info()

Lets remove the trainling space from EncodedPixels column:

In [ ]:
mask_rle_df = mask_rle_df.rename(columns={' EncodedPixels': 'EncodedPixels'})
mask_rle_df.info()

There are images with multiple EncodedPixels in separate rows, lets just merge them group ny ImageId

In [ ]:
img_mask_df = mask_rle_df[mask_rle_df['EncodedPixels'] != ' -1'].groupby('ImageId').count()
img_mask_df.head()

In [ ]:
img_mask_df = img_mask_df.rename(columns={'EncodedPixels': 'NMasks'})
img_mask_df['EncodedMasks']=''
img_mask_df.head()

In [ ]:
i = 0
for index, row in img_mask_df.iterrows():
    #print(imageid)
    rles = mask_rle_df.loc[mask_rle_df['ImageId']==index, 'EncodedPixels'].values
    #print(rles)
    img_mask_df.at[index,'EncodedMasks'] = rles
    i = i+1
print('Total image: ', i)

We also have images without mask, but the mask files still need to be created as well. Lets get the list then:

In [ ]:
img_nomask_df = mask_rle_df[mask_rle_df['EncodedPixels'] == ' -1'].groupby('ImageId').sum()
img_nomask_df.head()

Now, lets write the mask of each image info files:

In [ ]:
import cv2
import zipfile

In [ ]:
OUTPUT = '../working/masks/'
if not os.path.exists(OUTPUT):
    os.mkdir(OUTPUT)

In [ ]:
out_size = (256,256)
i = 0
j = 0
with zipfile.ZipFile(os.path.join(OUTPUT, 'mask.zip'), 'w') as zip:
    for index, row in img_mask_df.iterrows():
        rles = row['EncodedMasks']
        mask = np.zeros((1024,1024))
        for rle in rles:
            mask = mask + rle2mask(rle, 1024,1024).T
        file = index+'.png'
        _, png = cv2.imencode('.png', cv2.resize(mask, out_size))
        zip.writestr(file, png)
        i = i + 1
    for index, row in img_nomask_df.iterrows():
        mask = np.zeros((1024,1024))
        file = index+'.png'
        _, png = cv2.imencode('.png', cv2.resize(mask, out_size))
        zip.writestr(file, png)
        j = j + 1
print('Total mask files written: ', i+j)

In [ ]:
print(os.listdir('../working/masks'))